Q1. Using the URDF file, determine the screw axes (Si ∈ R 6 or Bi ∈ R 6) of the robot joints when the robot is at its home/zero configuration. Note that a fixed space frame {s} is attached to the robot base, and a body frame {b} is attached to the robot end-effector. To have a better understanding of the pose of the robot axes at its home/zero configuration, import the URDF file into Python using the function load urdf in pytransform3d package (Fig. 1). For more details, refer to the sample code in the folder assigned to your group.

In [30]:
import numpy as np
from scipy.spatial.transform import Rotation

joints_data = [
    {
        'name': 'joint1',
        'parent': 'link_base',
        'child': 'link1',
        'origin': [0, 0, 0.267],
        'rpy': [0, 0, 0],
        'axis': [0, 0, 1]  
    },
    {
        'name': 'joint2',
        'parent': 'link1',
        'child': 'link2',
        'origin': [0, 0, 0],
        'rpy': [-1.5708, 0, 0],  
        'axis': [0, 0, 1]
    },
    {
        'name': 'joint3',
        'parent': 'link2',
        'child': 'link3',
        'origin': [0, -0.293, 0],
        'rpy': [1.5708, 0, 0],
        'axis': [0, 0, 1]
    },
    {
        'name': 'joint4',
        'parent': 'link3',
        'child': 'link4',
        'origin': [0.0525, 0, 0],
        'rpy': [1.5708, 0, 0], 
        'axis': [0, 0, 1]
    },
    {
        'name': 'joint5',
        'parent': 'link4',
        'child': 'link5',
        'origin': [0.0775, -0.3425, 0],
        'rpy': [1.5708, 0, 0], 
        'axis': [0, 0, 1]
    },
    {
        'name': 'joint6',
        'parent': 'link5',
        'child': 'link6',
        'origin': [0, 0, 0],
        'rpy': [1.5708, 0, 0],  
        'axis': [0, 0, 1]
    },
    {
        'name': 'joint7',
        'parent': 'link6',
        'child': 'link7',
        'origin': [0.076, 0.097, 0],
        'rpy': [-1.5708, 0, 0],  
        'axis': [0, 0, 1]
    }
]

def rpy_to_rotation_matrix(rpy):
    r = Rotation.from_euler('xyz', rpy)
    return r.as_matrix()

def transformation_matrix(origin, rpy):
    T = np.eye(4)
    T[:3, :3] = rpy_to_rotation_matrix(rpy)
    T[:3, 3] = origin
    return T

def skew_symmetric(v):
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def compute_screw_axis(omega, q):
    
    omega = np.array(omega)
    q = np.array(q)
    v = -np.cross(omega, q)
    return np.concatenate([v, omega])

def adjoint_transformation(T):
    R = T[:3, :3]
    p = T[:3, 3]
    Ad = np.zeros((6, 6))
    Ad[:3, :3] = R
    Ad[3:, 3:] = R
    Ad[:3, 3:] = skew_symmetric(p) @ R
    return Ad

# ============================================================================
# FORWARD KINEMATICS - COMPUTE JOINT FRAMES IN SPACE FRAME
# ============================================================================

T_current = np.eye(4)  
joint_transforms = []  

for i, joint in enumerate(joints_data):
    T_joint = transformation_matrix(joint['origin'], joint['rpy'])
    
    T_current = T_current @ T_joint
    joint_transforms.append(T_current.copy())


M = T_current.copy()

# ============================================================================
# SPACE FRAME SCREW AXES (Si)
# ============================================================================

space_screws = []

for i, (joint, T) in enumerate(zip(joints_data, joint_transforms)):
    # Local axis in joint frame
    axis_local = np.array(joint['axis'])
    
    # Transform axis to space frame using rotation part of T
    R = T[:3, :3]
    omega_s = R @ axis_local
    omega_s = omega_s / np.linalg.norm(omega_s) 
    
    # Position of joint origin in space frame
    q_s = T[:3, 3]
    
    # Compute screw axis
    S_i = compute_screw_axis(omega_s, q_s)
    space_screws.append(S_i)



# ============================================================================
# BODY FRAME SCREW AXES (Bi)
# ============================================================================

M_inv = np.linalg.inv(M)
Ad_M_inv = adjoint_transformation(M_inv)

body_screws = []

for i, S_i in enumerate(space_screws):
    B_i = Ad_M_inv @ S_i
    body_screws.append(B_i)

print("\nSPACE FRAME SCREW AXES (Si):")
for i, S in enumerate(space_screws):
    print(f"   S{i+1} = {S}")

print("\nBODY FRAME SCREW AXES (Bi):")
for i, B in enumerate(body_screws):
    print(f"   B{i+1} = {B}")



SPACE FRAME SCREW AXES (Si):
   S1 = [-0. -0. -0.  0.  0.  1.]
   S2 = [-0.267     0.       -0.        0.        1.       -0.000004]
   S3 = [ 0.000001 -0.       -0.        0.        0.        1.      ]
   S4 = [ 0.56      0.       -0.0525    0.       -1.       -0.000004]
   S5 = [-0.000004  0.13      0.000001  0.        0.000007 -1.      ]
   S6 = [-0.2175   -0.000001  0.13      0.        1.        0.000011]
   S7 = [-0.000004  0.206     0.000002  0.        0.000007 -1.      ]

BODY FRAME SCREW AXES (Bi):
   B1 = [-0.000003 -0.206     0.000002  0.       -0.000007 -1.      ]
   B2 = [-0.1465    0.000002  0.206     0.       -1.        0.000011]
   B3 = [-0.000002 -0.206     0.000002  0.       -0.000007 -1.      ]
   B4 = [ 0.4395   -0.000001 -0.1535    0.        1.       -0.000004]
   B5 = [ 0.     0.076  0.     0.    -0.     1.   ]
   B6 = [-0.097    -0.        0.076     0.       -1.       -0.000004]
   B7 = [ 0.  0.  0.  0. -0.  1.]


2. (1/10) In order to make sure that the screw axes are determined correctly, use your forward kinematics
function (FK SpaceForm or FK BodyForm from HW#5) to compute the end-effector configuration Tsb ∈
SE(3) for an arbitrary set of feasible joint angles θ ∈ R n (refer to the Specification file to find the joint limits), and then, use the function plot transform (refer to the sample code) to visually verify that the end-effector configuration Tsb coincides with the end-effector {b}-frame of the robot imported into Python at the same configuration θ.

In [31]:
import numpy as np
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

np.set_printoptions(precision=6, suppress=True, linewidth=100)


try:
    from pytransform3d.urdf import UrdfTransformManager
    PYTRANSFORM3D_AVAILABLE = True
except ImportError:
    PYTRANSFORM3D_AVAILABLE = False
    print("Note: pytransform3d not available. Will use manual FK comparison.")

# ============================================================================
# FORWARD KINEMATICS IMPLEMENTATION
# ============================================================================


def screw_to_se3(S):
    """Convert screw axis S = [v; ω] to se(3) matrix [S]."""
    v = S[:3]
    omega = S[3:]
    se3_matrix = np.zeros((4, 4))
    se3_matrix[:3, :3] = skew_symmetric(omega)
    se3_matrix[:3, 3] = v
    return se3_matrix

def matrix_exponential_se3(se3_matrix, theta):
    """Compute matrix exponential e^([S]θ) for SE(3)."""
    omega_hat = se3_matrix[:3, :3]
    v = se3_matrix[:3, 3]
    
    omega = np.array([omega_hat[2, 1], omega_hat[0, 2], omega_hat[1, 0]])
    omega_norm = np.linalg.norm(omega)
    
    if omega_norm < 1e-6:
        T = np.eye(4)
        T[:3, 3] = v * theta
        return T
    
    omega_hat_normalized = omega_hat / omega_norm
    theta_scaled = theta * omega_norm
    
    I = np.eye(3)
    R = I + np.sin(theta_scaled) * omega_hat_normalized + \
        (1 - np.cos(theta_scaled)) * (omega_hat_normalized @ omega_hat_normalized)
    
    omega_normalized = omega / omega_norm
    p = (I * theta_scaled + (1 - np.cos(theta_scaled)) * omega_hat_normalized + \
         (theta_scaled - np.sin(theta_scaled)) * (omega_hat_normalized @ omega_hat_normalized)) @ \
        (v / omega_norm)
    
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = p
    return T

def FK_SpaceForm(theta, S_matrix, M):
    """Forward kinematics: T_sb = e^([S1]θ1) ... e^([Sn]θn) M"""
    T = np.eye(4)
    n_joints = S_matrix.shape[1]
    
    for i in range(n_joints):
        S_i = S_matrix[:, i]
        se3_i = screw_to_se3(S_i)
        T = T @ matrix_exponential_se3(se3_i, theta[i])
    
    T_sb = T @ M
    return T_sb

# ============================================================================
# COMPUTE SCREW AXES AND M FROM URDF
# ============================================================================

def rpy_to_rotation_matrix(rpy):
    r = Rotation.from_euler('xyz', rpy)
    return r.as_matrix()

def transformation_matrix(origin, rpy):
    T = np.eye(4)
    T[:3, :3] = rpy_to_rotation_matrix(rpy)
    T[:3, 3] = origin
    return T

def compute_screw_parameters():
    """Compute S matrix and M from URDF joint definitions."""
    joints_data = [
        {'origin': [0, 0, 0.267], 'rpy': [0, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0, 0, 0], 'rpy': [-1.5708, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0, -0.293, 0], 'rpy': [1.5708, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0.0525, 0, 0], 'rpy': [1.5708, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0.0775, -0.3425, 0], 'rpy': [1.5708, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0, 0, 0], 'rpy': [1.5708, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0.076, 0.097, 0], 'rpy': [-1.5708, 0, 0], 'axis': [0, 0, 1]}
    ]
    
    T_current = np.eye(4)
    space_screws = []
    
    for joint in joints_data:
        T_joint = transformation_matrix(joint['origin'], joint['rpy'])
        T_current = T_current @ T_joint
        
        R = T_current[:3, :3]
        omega_s = R @ np.array(joint['axis'])
        omega_s = omega_s / np.linalg.norm(omega_s)
        
        q_s = T_current[:3, 3]
        v_s = -np.cross(omega_s, q_s)
        S_i = np.concatenate([v_s, omega_s])
        space_screws.append(S_i)
    
    M = T_current.copy()
    return np.column_stack(space_screws), M

S_matrix, M = compute_screw_parameters()

# ============================================================================
# TEST CONFIGURATION
# ============================================================================

# Define test joint angles (in radians)
theta_test = np.array([
    np.deg2rad(30),   # Joint 1: 30°
    np.deg2rad(45),   # Joint 2: 45°
    np.deg2rad(-30),  # Joint 3: -30°
    np.deg2rad(60),   # Joint 4: 60°
    np.deg2rad(45),   # Joint 5: 45°
    np.deg2rad(-30),  # Joint 6: -30°
    np.deg2rad(45)    # Joint 7: 45°
])

print("="*80)
print("FORWARD KINEMATICS COMPARISON WITH URDF")
print("="*80)
print(f"\nTest Configuration (degrees): {np.rad2deg(theta_test)}")

# Compute FK using our implementation
T_computed = FK_SpaceForm(theta_test, S_matrix, M)

print("\nComputed FK Result (T_sb):")
print(T_computed)
print(f"\nEnd-effector position: {T_computed[:3, 3]}")

# ============================================================================
# COMPARE WITH URDF IF PYTRANSFORM3D IS AVAILABLE
# ============================================================================

if PYTRANSFORM3D_AVAILABLE:
    print("\n" + "="*80)
    print("LOADING URDF MODEL FOR COMPARISON")
    print("="*80)
    
    try:
        urdf_path = "path/to/your/urdf/file.urdf"  # UPDATE THIS PATH
        
        tm = UrdfTransformManager()
        with open(urdf_path, 'r') as f:
            robot_urdf = f.read()
        tm.load_urdf(robot_urdf)
        
        # Set joint angles
        joint_names = ['joint1', 'joint2', 'joint3', 'joint4', 
                      'joint5', 'joint6', 'joint7']
        for i, joint_name in enumerate(joint_names):
            tm.set_joint(joint_name, theta_test[i])
        
        # Get end-effector transformation from URDF
        T_urdf = tm.get_transform('link7', 'link_base')
        
        print("\nURDF FK Result:")
        print(T_urdf)
        print(f"\nEnd-effector position: {T_urdf[:3, 3]}")
        
        print("\n" + "="*80)
        print("VERIFICATION RESULTS")
        print("="*80)
        
        diff = np.abs(T_computed - T_urdf)
        print("\nAbsolute Difference:")
        print(diff)
        
        pos_error = np.linalg.norm(T_computed[:3, 3] - T_urdf[:3, 3])
        rot_error = np.linalg.norm(T_computed[:3, :3] - T_urdf[:3, :3], 'fro')
        
        print(f"\nPosition Error: {pos_error:.2e} meters")
        print(f"Rotation Error (Frobenius norm): {rot_error:.2e}")
        print(f"Max Element Difference: {np.max(diff):.2e}")
        
        if np.allclose(T_computed, T_urdf, atol=1e-6):
            print("\n✓ SUCCESS: FK computation matches URDF model!")
        else:
            print("\n✗ WARNING: FK computation differs from URDF model")
            print("  Check screw axes calculation or M matrix")
        
        # Visualize comparison
        fig = plt.figure(figsize=(14, 6))
        
        # 3D visualization
        ax1 = fig.add_subplot(121, projection='3d')
        
        # Plot frames
        def plot_frame(ax, T, label, scale=0.1, alpha=1.0):
            origin = T[:3, 3]
            R = T[:3, :3]
            colors = ['r', 'g', 'b']
            for i in range(3):
                axis = R[:, i]
                ax.quiver(origin[0], origin[1], origin[2],
                         axis[0], axis[1], axis[2],
                         color=colors[i], arrow_length_ratio=0.3,
                         linewidth=2.5, alpha=alpha, length=scale,
                         label=f'{label}-{["X","Y","Z"][i]}' if i == 0 else '')
        
        # Plot base
        T_base = np.eye(4)
        plot_frame(ax1, T_base, 'Base', scale=0.15, alpha=0.5)
        
        # Plot computed FK
        plot_frame(ax1, T_computed, 'FK (Computed)', scale=0.12, alpha=0.9)
        
        # Plot URDF FK
        plot_frame(ax1, T_urdf, 'FK (URDF)', scale=0.10, alpha=0.6)
        
        ax1.set_xlabel('X (m)')
        ax1.set_ylabel('Y (m)')
        ax1.set_zlabel('Z (m)')
        ax1.set_title('End-Effector Frame Comparison\n(Frames should overlap)')
        ax1.legend()
        
        # Error plot
        ax2 = fig.add_subplot(122)
        ax2.axis('off')
        
        summary = f"""
VERIFICATION SUMMARY
{'='*50}

Test Configuration:
  θ = {np.rad2deg(theta_test).round(2)}°

Computed FK Position:
  x = {T_computed[0,3]:.6f} m
  y = {T_computed[1,3]:.6f} m
  z = {T_computed[2,3]:.6f} m

URDF FK Position:
  x = {T_urdf[0,3]:.6f} m
  y = {T_urdf[1,3]:.6f} m
  z = {T_urdf[2,3]:.6f} m

Errors:
  Position:  {pos_error:.2e} m
  Rotation:  {rot_error:.2e}
  Max Diff:  {np.max(diff):.2e}

Status: {'✓ PASSED' if pos_error < 1e-6 else '✗ FAILED'}

{'Screw axes are correctly determined!' if pos_error < 1e-6 else 'Check screw axes or M matrix.'}
"""
        
        ax2.text(0.1, 0.5, summary, fontsize=10, family='monospace',
                verticalalignment='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig('xarm7_urdf_comparison.png', dpi=150)
        print("\nVisualization saved as 'xarm7_urdf_comparison.png'")
        plt.show()
        
    except FileNotFoundError:
        print("\nError: URDF file not found. Please update 'urdf_path' variable.")
        print("You can still use the FK computation without URDF comparison.")
    except Exception as e:
        print(f"\nError loading URDF: {e}")
        print("Continuing with FK computation only.")

else:
    print("\n" + "="*80)
    print("PYTRANSFORM3D NOT AVAILABLE")
    print("="*80)
    print("\nTo enable URDF comparison, install pytransform3d:")
    print("  pip install pytransform3d")
    print("\nYou can still verify FK using the standalone verification script.")

# ============================================================================
# MANUAL VERIFICATION FUNCTION
# ============================================================================

def manual_urdf_fk(theta):
    """Manual FK computation following URDF structure exactly."""
    joints_data = [
        {'origin': [0, 0, 0.267], 'rpy': [0, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0, 0, 0], 'rpy': [-1.5708, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0, -0.293, 0], 'rpy': [1.5708, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0.0525, 0, 0], 'rpy': [1.5708, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0.0775, -0.3425, 0], 'rpy': [1.5708, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0, 0, 0], 'rpy': [1.5708, 0, 0], 'axis': [0, 0, 1]},
        {'origin': [0.076, 0.097, 0], 'rpy': [-1.5708, 0, 0], 'axis': [0, 0, 1]}
    ]
    
    T = np.eye(4)
    for i, joint in enumerate(joints_data):
        T_fixed = transformation_matrix(joint['origin'], joint['rpy'])
        axis = np.array(joint['axis'])
        R_joint = Rotation.from_rotvec(axis * theta[i]).as_matrix()
        T_joint = np.eye(4)
        T_joint[:3, :3] = R_joint
        T = T @ T_fixed @ T_joint
    
    return T

# Always do manual verification
print("\n" + "="*80)
print("MANUAL VERIFICATION (Direct URDF FK)")
print("="*80)

T_manual = manual_urdf_fk(theta_test)
print("\nManual URDF FK Result:")
print(T_manual)

print("\nComparison with Screw-based FK:")
diff_manual = np.abs(T_computed - T_manual)
print(f"Position Error: {np.linalg.norm(T_computed[:3, 3] - T_manual[:3, 3]):.2e} m")
print(f"Max Difference: {np.max(diff_manual):.2e}")
print(f"\nMatch: {np.allclose(T_computed, T_manual, atol=1e-6)}")

if np.allclose(T_computed, T_manual, atol=1e-6):
    print("\n✓✓✓ SUCCESS: Screw axes are correctly determined! ✓✓✓")
else:
    print("\n✗ Warning: Results don't match - check screw axes")

print("\n" + "="*80)

FORWARD KINEMATICS COMPARISON WITH URDF

Test Configuration (degrees): [ 30.  45. -30.  60.  45. -30.  45.]

Computed FK Result (T_sb):
[[-0.074008 -0.719666  0.690365  0.519981]
 [-0.779628 -0.389918 -0.490045 -0.050894]
 [ 0.621854 -0.574495 -0.532215  0.175699]
 [ 0.        0.        0.        1.      ]]

End-effector position: [ 0.519981 -0.050894  0.175699]

LOADING URDF MODEL FOR COMPARISON

Error: URDF file not found. Please update 'urdf_path' variable.
You can still use the FK computation without URDF comparison.

MANUAL VERIFICATION (Direct URDF FK)

Manual URDF FK Result:
[[-0.074008 -0.719666  0.690365  0.519981]
 [-0.779628 -0.389918 -0.490045 -0.050894]
 [ 0.621854 -0.574495 -0.532215  0.175699]
 [ 0.        0.        0.        1.      ]]

Comparison with Screw-based FK:
Position Error: 1.36e-16 m
Max Difference: 2.22e-16

Match: True

✓✓✓ SUCCESS: Screw axes are correctly determined! ✓✓✓

